In [1]:
import torch 
import torch.nn as nn


In [2]:

STENCIL_SIZE = 20
FEATURE_NUM = 11

input_features = torch.rand(STENCIL_SIZE, FEATURE_NUM)

rotation_inv_features =input_features[:,4:]

y = torch.rand(stencil_size, 2)

In [3]:
class MLP(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(MLP, self).__init__()
            self.input_size = input_size
            self.hidden_size1  = hidden_size1
            self.hidden_size2 = hidden_size2
            self.output_size = output_size
            self.linear1 = nn.Linear(self.input_size, self.hidden_size1)
            self.relu1 = nn.ReLU()
            self.linear2 = nn.Linear(self.hidden_size1, self.hidden_size2)
            self.relu2 = nn.ReLU()
            self.linear3 = nn.Linear(self.hidden_size2, self.output_size)
            self.relu3 = nn.ReLU()
            
        def forward(self, x):
            hidden1 = self.linear1(x)
            relu1  =self.relu1(hidden1)
            hidden2 = self.linear2(relu1)
            relu2 = self.relu2(hidden2)
            hidden3 = self.linear3(relu2)
            output = self.relu3(hidden3)
            return output

In [4]:
input_size = rotation_inv_features.size()[1]
hidden_size1 = 32
hidden_size2 = 64
output_size = 64
embeddingNetwork = MLP(input_size,hidden_size1, hidden_size2, output_size)
loss = torch.nn.BCELoss()
optimizer = torch.optim.SGD(embeddingNetwork.parameters(), lr = 0.01)

In [7]:
G = embeddingNetwork(rotation_inv_features)

In [14]:
G_split_size = 4
G_ = G[:, :G_split_size]

In [15]:
L = torch.matmul(torch.transpose(G, 1,0), input_features)
L_ = torch.matmul(torch.transpose(input_features, 1, 0), G_)

In [16]:
D = torch.matmul(L, L_)

In [17]:
D_flattened = D.flatten()

In [18]:
input_size = D_flattened.size()[0]
hidden_size1 = 128
hidden_size2 = 64
output_size = 2
fittingNetwork = MLP(input_size, hidden_size1, hidden_size2, output_size)

optimizer = torch.optim.SGD(fittingNetwork.parameters(), lr = 0.01)

In [19]:
res = fittingNetwork(D_flattened)

In [30]:
a, b = res.detach().numpy()


In [31]:
a

0.34391254